In [95]:
from colour import Color
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydeck as pdk
import geopandas as gpd
from shapely import wkt

In [96]:
df = pd.read_parquet("../data/metro_regions.parquet")
df = gpd.GeoDataFrame(df)
df["geometry"] = df["geometry"].apply(wkt.loads)
df = df.set_geometry("geometry")
df.crs = "EPSG:3857"
df.to_crs(epsg=4326, inplace=True)

In [109]:
without_geo = pd.DataFrame(df.drop(columns=["geometry"]))
with_geo = pd.concat(
    [
        without_geo,
        df["geometry"].y,
        df["geometry"].x,
    ],
    axis=1,
)
with_geo.rename(columns={0: "latitude", 1: "longitude"}, inplace=True)
with_geo.head()

with_geo.to_csv("../data/metro_regions.csv", index=False)

In [110]:
with_geo.head()

,metro,country_code,population,cities,latitude,longitude
0,Shenzhen,CN,36825407,"[Nam Cheong, Wan Chai, Tseung Kwan O, Tai Koo,...",22.54554,114.06830
1,Tokyo,JP,23736774,"[Oami, Higashikurume, Wako, Ushiku, Togane, Zu...",35.68950,139.69171
2,Langfang,CN,30051058,"[Tianjin, Beijing]",39.52079,116.71471
3,Shanghai,CN,4165757,"[Songjiang, Huilong, Taicang, Zhabei, Jiashan,...",31.22222,121.45806
4,New York City,US,16427587,"[The Bronx, Woodhaven, Hoboken, Wakefield, Pis...",40.71427,-74.00597


In [35]:
def get_dataset(csv_name):
    dataset = pd.read_csv(csv_name, delimiter=";")
    dataset = dataset.drop_duplicates()

    # pick columns to use
    dataset["Latitude"] = dataset["Coordinates"].str.split(",").str[0].astype(float)
    dataset["Longitude"] = dataset["Coordinates"].str.split(",").str[1].astype(float)
    columns = [
        "Geoname ID",
        "Name",
        "Population",
        "Country name EN",
        "Latitude",
        "Longitude",
    ]
    dataset = dataset[columns]
    return dataset

dataset = get_dataset("geonames.csv")
len(dataset[dataset["Country name EN"] == "United States"])

,geoname_id,name,population,cou_name_en,latitude,longitude
2100,12156436,Bir Hadaj,5735,Israel,34.70857,31.02416
2101,1252699,Zahirābād,46504,India,77.60743,17.68138
2102,1253621,Ūttukkuli,7508,India,77.45431,11.16892
